In [1]:
from common import *

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:54: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


## Data Module

In [2]:
root_data_dir = "/workspaces/HIAAC-KR-Dev-Container/shared_data/seam_ai_datasets/seam_ai/images"
root_annotation_dir = "/workspaces/HIAAC-KR-Dev-Container/shared_data/seam_ai_datasets/seam_ai/annotations"


data_module = GenericDataModule(
    root_data_dir=root_data_dir,
    root_annotation_dir=root_annotation_dir,
    transforms=[
        TransformPipeline([
            SwapAxes(0, -1),
            SelectChannel(0),
            # SwapAxes(0, 1),
            PadCrop(1008, 784, padding_mode="reflect", seed=42, constant_values=0),
            RepeatChannel(repeats=3, axis=0),
            CastTo(np.float32),
        ]), 

        TransformPipeline([
            # SwapAxes(0, -1),
            PadCrop(1008, 784, padding_mode="reflect", seed=42, constant_values=0),
            CastTo(np.int64),
        ]), 
    ],
    batch_size=1,
    num_workers=1
)

data_module

DataModule
    Data: /workspaces/HIAAC-KR-Dev-Container/shared_data/seam_ai_datasets/seam_ai/images
    Annotations: /workspaces/HIAAC-KR-Dev-Container/shared_data/seam_ai_datasets/seam_ai/annotations
    Batch size: 1

In [3]:
data_module.setup("fit")
batch = next(iter(data_module.train_dataloader()))
print(batch[0].shape, batch[1].shape)

torch.Size([1, 3, 1008, 784]) torch.Size([1, 1, 1008, 784])


## Model

In [4]:
from minerva.models.ssl.dinov2 import (
    DinoVisionTransformer,
    SETR_PUP,
    NestedTensorBlock,
    MemEffAttention,
    DinoV2
)
from functools import partial

backbone = DinoVisionTransformer(
    patch_size=14,
    embed_dim=384,
    depth=12,
    num_heads=6,
    mlp_ratio=4,
    block_fn=partial(NestedTensorBlock, attn_class=MemEffAttention),  # type: ignore
    init_values=1e-5,
    block_chunks=0,
)

head = SETR_PUP(embedding_dim=384, num_classes=6)

Not using xFormers lib!


In [5]:
model = DinoV2(
    backbone=backbone,
    head=head,
    loss_fn=torch.nn.CrossEntropyLoss(),
    output_shape = (1008, 784),
    middle=False
)

model

DinoV2(
  (backbone): DinoVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0-11): 12 x NestedTensorBlock(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): MemEffAttention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
     

In [6]:
ckpt_file = "/workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/docs/notebooks/examples/seismic/facies_classification/f3/dinov2_vits14_pretrain.pth"
ckpt = torch.load(ckpt_file, map_location="cpu")

/tmp/ipykernel_89260/1669039306.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_file, map_location="cpu")


## Trainer

In [7]:
log_dir = "./logs"
logger = CSVLogger(log_dir, name="dinov2", version="parihaka")
checkpoint = ModelCheckpoint(
    save_top_k=1,
    save_last=True,
)


trainer = L.Trainer(
    max_epochs=3,
    limit_train_batches=3,
    limit_val_batches=3,
    limit_test_batches=3,
    limit_predict_batches=3,
    accelerator="gpu",
    devices=1,
    logger=logger,
    callbacks=[checkpoint],
)

pipeline = SimpleLightningPipeline(
    model=model,
    trainer=trainer,
    log_dir=log_dir + "/f3_segmentation",
    save_run_status=True
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/lightning/fabric/utilities/seed.py:42: No seed found, seed set to 0
Seed set to 0


Log directory set to: /workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/docs/notebooks/examples/seismic/facies_classification/parihaka/logs/f3_segmentation


In [8]:
pipeline.run(data_module, task="fit")

/usr/local/lib/python3.10/dist-packages/lightning/fabric/loggers/csv_logs.py:268: Experiment logs directory ./logs/dinov2/parihaka exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory ./logs/dinov2/parihaka/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/docs/notebooks/examples/seismic/facies_classification/parihaka/logs/f3_segmentation/run_2024-12-05-02-39-13a494413c7d074f13a0d7d80ffd609b0a.yaml



  | Name     | Type                  | Params | Mode 
-----------------------------------------------------------
0 | backbone | DinoVisionTransformer | 22.1 M | train
1 | head     | SETR_PUP              | 915 K  | train
2 | loss_fn  | CrossEntropyLoss      | 0      | train
-----------------------------------------------------------
23.0 M    Trainable params
0         Non-trainable params
23.0 M    Total params
91.887    Total estimated model params size (MB)
247       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 2: 100%|██████████| 3/3 [00:01<00:00,  2.88it/s, v_num=haka]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 3/3 [00:13<00:00,  0.22it/s, v_num=haka]
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/docs/notebooks/examples/seismic/facies_classification/parihaka/logs/f3_segmentation/run_2024-12-05-02-39-13a494413c7d074f13a0d7d80ffd609b0a.yaml


In [9]:
r = trainer.predict(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 12.06it/s]


In [10]:
r[0].shape

torch.Size([1, 6, 1008, 784])